In [9]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import defaultdict
import copy

from multiprocessing import Process, Queue, Manager
import time
import sys


In [2]:
dict_path = Path(Path.cwd(),'repos/word_squares/top10k.txt')
words = Path(dict_path).open('r').read().split('\n')
words = [word.lower() for word in words]
print(f'{len(words)} | {words[:10]}')

9999 | ['you', 'i', 'to', 'the', 'and', 'that', 'of', 'me', 'in', 'this']


In [3]:
def add_word_to_tree(tree, word):
    if len(word) > 0:
        try:
            sub_tree = tree[word[0]]
        except:
            sub_tree = defaultdict(lambda: defaultdict())
        tree[word[0]] = add_word_to_tree(sub_tree, word[1:])
        return tree
    else:
        return defaultdict(lambda: defaultdict())


In [4]:
char_tree = defaultdict(lambda: defaultdict())

for word in words:
    char_tree = add_word_to_tree(char_tree, word)


In [5]:
char_tree['t']['h']['i']

defaultdict(None,
            {'s': defaultdict(<function __main__.add_word_to_tree.<locals>.<lambda>()>,
                         {}),
             'n': defaultdict(<function __main__.add_word_to_tree.<locals>.<lambda>()>,
                         {'k': defaultdict(None,
                                      {'i': defaultdict(<function __main__.add_word_to_tree.<locals>.<lambda>()>,
                                                   {'n': defaultdict(None,
                                                                {'g': defaultdict(<function __main__.add_word_to_tree.<locals>.<lambda>()>,
                                                                             {})})})}),
                          'g': defaultdict(None,
                                      {'y': defaultdict(<function __main__.add_word_to_tree.<locals>.<lambda>()>,
                                                   {})}),
                          'n': defaultdict(None,
                                      {'

# Make Word Squares

In [115]:
def get_loc_from_index(i, n):
    x = i % n
    y = int(i / n)
    return [x, y]

def get_index_from_loc(loc, n):
    x, y = loc
    return y * n + x

def get_possible_chars(sq, loc, char_tree):
    x, y = loc
    partial_word1 = sq[x, :y]
    partial_word2 = sq[:x, y]

    options1 = get_possible_chars_from_partial_word(partial_word1, char_tree)
    options2 = get_possible_chars_from_partial_word(partial_word2, char_tree)
    return  options1.intersection(options2)

def get_possible_chars_from_partial_word(partial_word, char_tree):
    t = copy.copy(char_tree)
    for char in partial_word:
        if char in t.keys():
            t = t[char]
        else:
            t = {}
    return set(t.keys())

def get_char_tree(n):
    char_tree = defaultdict(lambda: defaultdict())
    for word in words:
        if len(word) == n:
            char_tree = add_word_to_tree(char_tree, word)
    return char_tree


class Square():
    def __init__(self, sq):
        self.sq = copy.copy(sq)
        self.symmetry_score = self.get_symmetry()
    
    def __eq__(self, other): 
        if np.all(self.sq == other.sq) or np.all(self.sq == other.sq.T): 
            return True
        else: 
            return False
    
    def __ge__(self, other):
        return (not __lt__(self, other) and not __eq__(self, other))
    
    def __lt__(self, other):
        return np.mean(self.sq < other.sq) > 0.5
    
    def __hash__(self):
        sq = self.sq if np.mean(self.sq > self.sq.T) > 0.5 else self.sq.T
        return hash(sq.tostring())
    
    def __str__(self):
        array_string = ''
        for line in self.sq:
            for char in line:
                array_string += f'{char} '
            array_string += f'\n'
        return array_string
    
    def get_symmetry(self):
        return np.mean(self.sq == self.sq.T)


def get_partial_squares(sq):
    sq = copy.copy(sq)
    i = np.sum(sq != '')
    n = sq.shape[0]
    loc = get_loc_from_index(i, n)
    x, y = loc
    possible_chars = get_possible_chars(sq, loc, char_tree)

    sqs = []

    if len(possible_chars) == 0:
        return []

    for char in possible_chars:
        sq[x, y] = char

        if i < n**2 - 1:
            sqs.extend(get_partial_squares(sq))
        else:
            sqs.extend([Square(sq)])

    return sqs

def get_squares(n):

    char_tree = get_char_tree(n)
    
    sq = np.chararray((n, n))
    sq.fill('')
    sq = sq.astype('<U1')

    _start = time.time()

    sqs = get_partial_squares(sq)

    print("Took {0} seconds".format((time.time() - _start)))

    return sqs

def get_unique_sqs(sqs):
    return list(set(sqs))

def print_sqs(sqs):
    for sq in sqs:
        print(sq)

In [116]:
x = get_squares(3)
unique_sqs = get_unique_sqs(x)
unique_sqs.sort(key=lambda x: x.symmetry_score, reverse=False)
print(len(unique_sqs))

Took 2.5752923488616943 seconds
15053


In [117]:
print_sqs(unique_sqs[:2])

s e e 
p a w 
a r e 

i t s 
c a p 
e r a 



# First level parallel

In [ ]:

def get_partial_squares(sq, limit=None):

    sq = copy.copy(sq)
    n = sq.shape[0]
    i = np.sum(sq != '')

    if limit is None:
        limit = n**2 - 1

    loc = get_loc_from_index(i, n)
    x, y = loc
    possible_chars = get_possible_chars(sq, loc, char_tree)

    sqs = []

    if len(possible_chars) == 0:
        return []

    for char in possible_chars:

        sq[x, y] = char

        if i < limit:
            sqs.extend(get_partial_squares(sq))
        else:
            sqs.extend([Square(sq)])
    
    return sqs


def get_parallel_partial_squares(sq, done_list):

    sq = copy.copy(sq)
    n = sq.shape[0]
    i = np.sum(sq != '')

    loc = get_loc_from_index(i, n)
    x, y = loc
    possible_chars = get_possible_chars(sq, loc, char_tree)

    sqs = []

    if len(possible_chars) == 0:
        return []

    for char in possible_chars:
        sq[x, y] = char

        sqs = get_partial_squares(sq)
        done_list.append(sqs)
    
    return done_list

def get_squares_parallel(n, max_processes = 80):
    with Manager() as manager:

        done_list = manager.list() 

        global char_tree
        char_tree = get_char_tree(n)
        
        sq = np.chararray((n, n))
        sq.fill('')
        sq = sq.astype('<U1')

        processes_level = 0
        possible_chars = get_possible_chars(sq, [0,0], char_tree)
        partial_sqs = get_partial_squares(sq, limit=processes_level)
        
        procs = []
        _start = time.time()

        print(f'Starting {len(partial_sqs)} processes')
        for sq in partial_sqs:
          
            # print(name)
            proc = Process(target=get_parallel_partial_squares, args=(sq.sq, done_list))
            proc.daemon = True
            procs.append(proc)
            proc.start()

        # complete the processes
        for proc in procs:
            x = proc.join()

        print("Took {0} seconds".format((time.time() - _start)))
    
        return list(done_list), partial_sqs


In [123]:
x, y = get_squares_parallel(3)
print(len(x))
unique_sqs = get_unique_sqs(x)
unique_sqs.sort(key=lambda x: x.symmetry_score, reverse=False)
print(len(unique_sqs))

Starting 24 processes
Took 0.5361182689666748 seconds
137


TypeError: unhashable type: 'list'

In [94]:
y[3].sq

chararray([['y', '', ''],
           ['', '', ''],
           ['', '', '']], dtype='<U1')

In [81]:
x[4][0].sq

chararray([['y', 'e', 't'],
           ['o', 'r', 'e'],
           ['u', 'r', 'n']], dtype='<U1')

In [72]:
print_sqs(unique_sqs)

y a p 
e r a 
s e w 

y e s 
a r k 
p a y 

y a p 
e g o 
s e t 

y e s 
a g e 
p o t 

y a p 
e r r 
s k y 

y o u 
e r r 
t e n 

y e t 
a r e 
p a n 

y a p 
e r a 
t e n 

y a p 
e r a 
s k y 

y e t 
o r e 
u r n 

y e s 
a r k 
p r y 

y e s 
a r e 
p a w 

y e s 
e r a 
t r y 

y e s 
a w e 
p e t 

y a p 
e w e 
t e e 

y e t 
a w e 
p e e 

y e t 
a g o 
p o p 

y a p 
e g o 
t o p 

y e t 
e r r 
s a y 

y a p 
e g o 
s o d 

y e s 
a g o 
p o d 

y a p 
e w e 
s e t 

y e s 
e r a 
t a g 

y e s 
e y e 
t e e 

y e t 
e g o 
s o y 

y e t 
e r a 
s a g 

y e t 
e v e 
s e e 

y a p 
a g e 
p o t 

y e s 
e v e 
t e e 

y a p 
a g o 
p e t 

y e t 
e y e 
s e e 

y e t 
e w e 
s e e 

y e s 
e w e 
t e e 

y e s 
e g o 
t o y 

y e t 
e w e 
t e e 

y e t 
e v e 
t e e 

y e s 
e y e 
s e x 

y e s 
e w e 
s e x 

y a p 
a p e 
p e p 

y a p 
a w e 
p e p 

y e s 
e r a 
s a y 

y a p 
a i r 
p r y 

y a p 
a l e 
p e p 

y e s 
e l k 
s k y 

y e t 
e y e 
t e e 

y o u 
o a

In [49]:
unique_sqs

# All Parallel

In [135]:
def get_char_tree(n):
    char_tree = defaultdict(lambda: defaultdict())
    for word in words:
        if len(word) == n:
            char_tree = add_word_to_tree(char_tree, word)
    return char_tree


def get_squares(n, processes = 6):
    with Manager() as manager:

        global char_tree
        char_tree = get_char_tree(n)
        
        sq = np.chararray((n, n)).astype('<U1')
        sq.fill('')

        work_queue = Queue()
        done_list = manager.list() 

        work_queue.put((0, sq))


        procs = []
        _start = time.time()

        for i in range(processes):
            # print(name)
            proc = Process(target=get_partial_squares, args=(work_queue, done_list))
            proc.daemon = True
            procs.append(proc)
            proc.start()
            sleep(0.01)     

            
        # complete the processes
        for proc in procs:
            proc.join()

        print("Took {1} seconds".format(count, 
            (time.time() - _start)))
    
        return list(done_list)


def get_partial_squares(work_queue, done_list):

    while not work_queue.empty():
        i, sq = work_queue.get()

        sq = copy.copy(sq)
        n = sq.shape[0]
        loc = get_loc_from_index(i, n)
        x, y = loc
        possible_chars = get_possible_chars(sq, loc, char_tree)

        sqs = []

        if len(possible_chars) == 0:
            continue

        for char in possible_chars:
            sq[x, y] = char

            if i < n**2 - 1:
                work_queue.put((i + 1, sq))
            else:
                done_list.extend([Square(sq)])
        
        sleep(0.000001)


In [137]:
sqs = get_squares(3)

KeyboardInterrupt: 

In [123]:
sqs

In [124]:
unique_sqs = get_unique_sqs(x)
print(len(unique_sqs))
unique_sqs.sort(key=lambda x: x.symmetry_score, reverse=False)
print_sqs(unique_sqs)

25
i s 
n o 

a s 
n o 

a t 
n o 

a t 
s o 

i t 
s o 

i n 
s o 

i s 
t o 

a n 
s o 

i n 
t o 

a n 
t o 

a s 
t o 

i t 
n o 

o n 
n o 

i n 
n o 

i t 
t o 

u s 
s o 

n o 
o r 

t o 
o r 

d o 
o r 

a s 
s o 

i s 
s o 

a n 
n o 

g o 
o r 

s o 
o r 

a t 
t o 



In [20]:

def reader_proc(queue):
    ## Read from the queue; this will be spawned as a separate Process
    while True:
        msg = queue.get()         # Read from the queue and do nothing
        if (msg == 'DONE'):
            break

def writer(count, queue):
    ## Write to the queue
    for ii in range(0, count):
        queue.put(ii)             # Write 'count' numbers into the queue
    queue.put('DONE')


pqueue = Queue() # writer() writes to pqueue from _this_ process
for count in [10**4, 10**5]:             
    ### reader_proc() reads from pqueue as a separate process
    reader_p = Process(target=reader_proc, args=((pqueue),))
    reader_p.daemon = True
    reader_p.start()        # Launch reader_proc() as a separate python process

    _start = time.time()
    writer(count, pqueue)    # Send a lot of stuff to reader()
    reader_p.join()         # Wait for the reader to finish
    print("Sending {0} numbers to Queue() took {1} seconds".format(count, 
        (time.time() - _start)))

Sending 10000 numbers to Queue() took 0.0828714370727539 seconds
Sending 100000 numbers to Queue() took 0.7374887466430664 seconds


In [101]:
char_tree = defaultdict(lambda: defaultdict())

for word in words:
    if len(word) == n:
        char_tree = add_word_to_tree(char_tree, word)


In [102]:
for i, char in enumerate(list(char_tree.keys())[:n**2]):
    loc = get_loc_from_index(i, n)
    x, y = loc
    sq[x, y] = char
sq

array([['t', 'w', 'f', 'd'],
       ['h', 'c', 's', 'v'],
       ['j', 'g', 'm', 'l'],
       ['y', 'b', 'o', 'e']], dtype='<U1')

In [16]:
x = [['i','t','e','m'],
    ['t','i','m','e'],
    ['e','m','i','t'],
    ['m','e','','']]
sq = np.array(x)
sq

array([['i', 't', 'e', 'm'],
       ['t', 'i', 'm', 'e'],
       ['e', 'm', 'i', 't'],
       ['m', 'e', '', '']], dtype='<U1')

In [18]:
np.sum(sq != '')

14

In [104]:
loc = [2, 1]
get_possible_chars(sq, loc)

TypeError: get_possible_chars() missing 1 required positional argument: 'char_tree'

In [105]:

x, y = loc
partial_word1 = sq[x, :y]
partial_word2 = sq[:x, y]
print(sq[x,y], partial_word1, partial_word2)

m ['e'] ['t' 'i']


In [106]:
char_tree

in__.add_word_to_tree.<locals>.<lambda>()>,
                                                                {}),
                                                    'g': defaultdict(<function __main__.add_word_to_tree.<locals>.<lambda>()>,
                                                                {}),
                                                    'm': defaultdict(<function __main__.add_word_to_tree.<locals>.<lambda>()>,
                                                                {})}),
                                       'e': defaultdict(None,
                                                   {'a': defaultdict(<function __main__.add_word_to_tree.<locals>.<lambda>()>,
                                                                {})}),
                                       'o': defaultdict(None,
                                                   {'t': defaultdict(<function __main__.add_word_to_tree.<locals>.<lambda>()>,
                                            